# Gaussian IP Efficencies:

Invesitgating improving the unitary gauss function to be faster



In [1]:
import numpy as np
def unitary_Gauss(x, center, FWHM):
    """
    Gaussian_function of area=1
	
	p[0] = A;
	p[1] = mean;
	p[2] = FWHM;
    """
    
    sigma = np.abs(FWHM) /( 2 * np.sqrt(2 * np.log(2)) )
    Amp = 1.0 / (sigma*np.sqrt(2*np.pi))
    tau = -((x - center)**2) / (2*(sigma**2))
    result = Amp * np.exp( tau )
    
    return result


In [12]:
x = np.arange(10, 100000)
center = 20400
FWHM = 50 

In [13]:
%%timeit 
for wav in x:
   result = unitary_Gauss(x, wav, FWHM)
    

1 loop, best of 3: 2min 51s per loop


In [ ]:
%%timeit 
for center in x:
    # in this case doing the sigma calulation twice
    result = (1.0 / ((np.abs(FWHM) /( 2 * np.sqrt(2 * np.log(2)) ))*np.sqrt(2*np.pi))) * np.exp( -((x - center)**2) / (2*((np.abs(FWHM) /( 2 * np.sqrt(2 * np.log(2)) ))**2)) )

In [8]:
%load_ext line_profiler
%lprun -f unitary_Gauss unitary_Gauss(x, c, fwhm)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [17]:
%%timeit  
# single line gaussian
for center in x:
    (1.0 / ((np.abs(FWHM) /( 2 * np.sqrt(2 * np.log(2)) ))*np.sqrt(2*np.pi))) * np.exp( -((x - center)**2) / (2*((np.abs(FWHM) /( 2 * np.sqrt(2 * np.log(2)) ))**2)) )

1 loop, best of 3: 7.07 s per loop


In [ ]:
%%timeit  
# single line gaussian
sigma = np.abs(FWHM) /( 2 * np.sqrt(2 * np.log(2)) );
for center in x:
    result = np.exp(-((x - center)**2) / (2*(sigma**2)))  / (sigma*np.sqrt(2*np.pi))

# Split up gausian calculation

Amplitude and sigma outside of loop to stop repition of calculations

In [14]:
def unitary_gaussian_constants(FWHM):
    sigma = np.abs(FWHM) / ( 2 * np.sqrt(2 * np.log(2)) )
    Amp = 1.0 / (sigma * np.sqrt(2 * np.pi))
    two_sigma_squared = 2 * (sigma**2)
    return Amp, two_sigma_squared

def unitary_gaussian_exp_part(x, center, Amp, two_sigma_squared):
    tau = -((x - center)**2) / two_sigma_squared
    return Amp * np.exp( tau )


In [16]:
%%timeit 
A, sigma22 = unitary_gaussian_constants(FWHM)
for center in x:
    result = unitary_gaussian_exp_part(x, center, A, sigma22)

1 loop, best of 3: 2min 58s per loop


In [17]:
%%timeit 
for center in x:
    A, sigma22 = unitary_gaussian_constants(FWHM)
    result = unitary_gaussian_exp_part(x, center, A, sigma22)

1 loop, best of 3: 2min 52s per loop


In [18]:
len(x)


99990

In [24]:
%timeit A, sigma22 = unitary_gaussian_constants(FWHM)   # cached result ?
A, sigma22 = unitary_gaussian_constants(FWHM)
%timeit result = unitary_gaussian_exp_part(x, center, A, sigma22)

The slowest run took 6.29 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.02 µs per loop
1000 loops, best of 3: 1.71 ms per loop


In [21]:
6.1 * 99990 /1e6
#seconds for constant

0.609939

In [25]:
1.7 *99990/1e3
   # seconds for exp

169.983